# 구조적  API 기본 연산

DataFrame은 Row 타입의 레코드(Record)와 각 레코드에 수행할 연산 표현식을 나타내는 여러 컬럼(Column)으로 구성됩니다. 스키마(Schema)는 각 컬럼명과 데이터타입을 정의합니다. DataFrame의 파티셔닝(Partitioning)은 DataFrame이나 Dataset이 클러스터에서 물리적으로 배치되는 형태를 정의합니다.

파티셔닝 스키마(Partitioning Schema)는 파티션을 배치하는 방법을 정의합니다. 파티셔닝의 분할 기준은 특정 컬럼이나 비결정론적인 값을 기반으로 설정할 수 있습니다.

우선 DataFrame을 생성합니다.

In [4]:
df = spark.read.format("json").load("./data/flight-data/json/2015-summary.json")

In [6]:
# DataFrame 은 컬럼을 가지며 스키마로 컬럼을 정의합니다.  
# DataFrame의 스키마를 봅시다,
df.printSchema()  #스키마는 관련된 모든 것을 하나로 묶는 역할을 합니다.

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



## 스키마
스키마는 DataFrame의 컬럼명과 데이터타입을 정의합니다. 데이터소스에서 스키마를 얻거나 직접 정의할 수 있습니다.
> 데이터를 읽기 전에 스키마를 정의해야하는지 여부는 상황에 따라 달라진다. 비정형 분석에서는 스키마-온-리드가 대부분 잘 동작합니다. 하지만 Long 데이터 타입을 Integer데이터 타입으로 잘못인식 하는 등 정밀도 문제가 발생할 수 있습니다. 따라서 운영환경에서 추출, 변환, 적재를 수행하는 ETL작업에 스파크를 이용한다면 직접 스키마를 정의해야한다. ETL작업중에 데이터타입을 알기 힘든 CSV나 JSON 등의 데이터소스를 사용하는 경우 스키마 푸론 과정에서 읽어들인 샘플 데이터의 타입에 따라 스키마를 결정할 수 있습니다.

In [7]:
spark.read.format("json").load("./data/flight-data/json/2015-summary.json").schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

스키마 여러 개의 `StructField` 타입 필드로 구성된  `StructType` 객체이다. `StructFiled` 는 이름, 데이터 타입, 컬럼 값이 없거나 `null`일 수 있는지 지정하는 불 값을 가진다.


필요한 경우 컬럼과 관련된 메타데이터를 지정할 수 있습니다. 메타데이터는 해당 컬럼과 관련된 정보이고, Spark의  머신러닝 라이브러리에서 사용합니다.

스키마는 복합데이터타입인 `StructType` 을 가질 수 있다. 다음은 DataFrame에 스키마를 만들고 적용하는 예제이다.

In [29]:
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema = StructType([
    StructField("DEST_COUNTRY_NAME", StringType(), True),
    StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
    StructField("count",LongType(), False, metadata={"hello":"world"})
])


# DataFrame에 스키마를 만들고 적용하는 예제
df = spark.read.format("json").schema(myManualSchema).load("./data/flight-data/json/2015-summary.json")


# 스파크는 자체 데이터 타입 정보를 사용하므로 프로그래밍 언어의 데이터타입을 스파크의 데이터 타입으로 설정할 수 없습니다.
# 다음절에는 스키마에 정의하는 컬럼에 대해 알아보겠습니다.


## 컬럼과 표현식
Spark의 컬럼은 스프레드시트, R의  DataFrame, Pandas 의 DataFrmae과 유사합니다. 사용자는 표현식으로 DataFrame의 컬럼을 선택 조작 제거할 수 있습니다.
Spark의 컬럼은 표현식으로 레코드 단위로 계산한 값을 단순히 나타내는 논리적인 구조입니다. 컬럼의 실젯값을 얻으려면 로우가 필요하고, 로우를 얻으려면 DataFrame이 필요합니다. DataFrame을 통하지 않으면 외부에서 컬럼에 접근할 수 없습니다.

컬럼 내용을 수정하려면 반드시 DataFrame의 Spark 트랜스포메이션을 사용해야합니다.

### 컬럼
컬럼을 생성하고 참조할 수 있는 다양한 방법이 있지만, `col` 함수나 `column` 함수를 사용하는 것이 가장 간단합니다. 이들 함수는 컬럼명을 인수로 받습니다.

In [30]:
from pyspark.sql.functions import coalesce,col

coalesce("comeColumnUsingColumn") # column 이라 나오는데 그거 없음.
col("someColumnNameUsingCol")
# https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.coalesce.html

Column<'someColumnNameUsingCol'>

이 책에서 col함수를 계속해서 사용합니다. 컬럼이 DataFrame에 있을지 없을지 알 수 없다. 컬럼은 컬럼명을 **카탈로그**에 저장된 정보와 비교하기 전까지 미확인 상태로 남습니다. 4장에서 알아본 것처럼 **분석기** 가 동작하는 단계에서 컬럼과 테이블을 분석합니다.


#### 명시적 컬럼 참조 
DataFrame의 컬럼은 `col` 메서드로 참조합니다. `col` 메소드는 조인 시 유용합니다. 예를 들어 DataFrame의 어떤 컬럼은 다른 DataFrame의  조인 컬럼 대상 칼럼에서 참조하기 위해 `col`메소드를 사용합니다. `col` 메소드를 사용해 명시적으로 컬럼을 정의하면 Spark는 분석기 실행 단계에서 컬럼 확인 절차를 생략합니다.

In [34]:
from pyspark.sql.functions import coalesce,col
df.select(col("count")).show()

+-----+
|count|
+-----+
|   15|
|    1|
|  344|
|   15|
|   62|
|    1|
|   62|
|  588|
|   40|
|    1|
|  325|
|   39|
|   64|
|    1|
|   41|
|   30|
|    6|
|    4|
|  230|
|    1|
+-----+
only showing top 20 rows



In [35]:
df

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

### 표현식
DataFrame을 정의할 때 컬럼을 표현식이라 했다. 표현식은 DataFrame 레코드의 여러 값에 대한 트랜스포메이션 집합을 의미한다.

여러 컬럼명이나 입력으로 박아 식별하고, 단일 값을 만들기 위해 다양한 표현식을 각 레코드에 적용하는 함수라 생각할 수 있다. 여기서 단일 값은 Map이나 Array같은 복합 데이터 타입일 수 있다.

표현식은 `expr`함수로 가장 간단한 사용할 수 있다. 이 함수를 이용해 DataFrame의 컬럼을 참조할 수 있다. 예를 들어 `expr("someCol")`은 `col("someCol")` 구문과 동일하게 동작한다.

#### 표현식으로 컬럼 표현
컬럼은 표현식의 일부 기능을 제공한다. `col()`함수를 호출해 트랜스포메이션을 수행하려면 반드시 컬럼 참조를 사용해야한다. `expr` 함수의 인수 표현식을 사용하면 표현식을 분석해 트랜스포메이션과 컬럼 참조를 알아낼 수 있고, 다음 트랜스포메이션에 컬럼 참조를 전달할 수 있다.

`expr("someCol -5")`, `col("somCol")-5`, `expr("someCol")-5` 는 모두 같은 트랜스포메이션 과정을 거칩니다. Spark과 연산 순서를 지정하는 논리적 트리로 컴파일하기 때문이다.

* 컬럼은 단지 표현식일 뿐이다.
* 컬럼과 컬럼의 트랜스포메이션은 파싱된 표현식과 동일한 논리적 실행 계획으로 컴파일된다.

```
(((col("someCol")+5)*200)-6)<col("otherCol")

```

In [40]:
from pyspark.sql.functions import expr
expr("(((someCol+5)*200)-6) < otherCol")

Column<'((((someCol + 5) * 200) - 6) < otherCol)'>

SQLdml `SELECT` 구문에 이진 표현식을 사용해도 잘 동작하며 동일한 결과글 생성합니다. 그 이유는 SQL 표현식과 위 예제의 DataFrame 코드는 실행시점에 동일한 논리 트리로 컴파일되기 떄문이다. 따라서 DataFrame 코드나  SQL로 표현식을 작성할 수 있으며 동일한 성능을 발휘한다.

#### DataFrame의 컬럼에 접근하기
* `printSchema` 메서드로 DataFrame 의 전체 정보를 확인할 수 있다. 
* 프로그래밍 방식으로 컬럼에 접근할 때는 DataFrame의 columns  속성을 사용합니다.

In [42]:
spark.read.format("json").load("./data/flight-data/json/2015-summary.json").columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

## 레코드와 로우
Spark 에서 DataFrame의 각 로우는 하나의 레코드이다. Spark는 레코드를 Row객체로 표현합니다. Spark는 값을 생성하기 위해 컬럼 표현식으로 Row 객체를 다룹니다.


Row 객체는 내부에 바이트 배열을 가집니다. 이 바이트 배열 인터페이스는 오직 컬럼표현식으로만 다룰 수 있으므로 사용자에게 노출되지 않습니다.


DataFrame을 사용해 드라이버에 개별 로우를 반환하는 명령은 하나 이상의 Row 타입을 반환합니다.
DataFrame의 `first` 메소드를 로우로 확인해봅니다.

In [43]:
df.first()

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

### 로우 생성하기
각 컬럼에 해당하는 값을 사용해 Row 객체를 직접 생성할 수 있습니다. Row 객체는 스키마 정보를 가지고 있지 않습니다. DataFrame만 유일하게 스키마를 가진다. 그러므로 Row객체를 직접 생성하려면 DataFrame의 스키마와 같은 순서로 값을 명시해야한다.

In [44]:
from pyspark.sql import Row
myRow = Row("Hello", None, 1, False)

로우의 데이터에 접근하는 방법은 원하는 위치를 지정하면 된다. 

In [45]:
myRow[0]

'Hello'

## DataFrame의 트랜스포메이션

DataFrame을 다루는 방법은 
* 로우나 컬럼 추가
* 로우나 컬럼 제거
* 로우를 컬럼으로 변환하거나 그 반대로 변환
* 컬럼값을 기준으로 로우 순서 변경
이 모든 유형의 작업은 트랜스포메이션으로 변환할 수 있다. 

### DataFrame 생성하기
원시 데이터소스에서 DataFrame을 생성할 수 있다. 그리고 생성한 DataFrame은 이 장 후반부에 SQL 쿼리를 실행하고 SQL 기본 트랜스포메이션을 확인하기 위해 임시 뷰로 등록하겠습니다.

In [47]:
df = spark.read.format("json").load("./data/flight-data/json/2015-summary.json")
df.createOrReplaceTempView("dfTable")

In [49]:
# Row 객체를 가진 Seq 타입을 직접 변환해 DataFrame 을 생성할 수 있다.
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType  , LongType

myManualSchema= StructType([
    StructField("some", StringType(), True),
      StructField("col", StringType(), True),
      StructField("names", LongType(), False)
])
myRow =Row("Hello", None, 1)
myDf = spark.createDataFrame([myRow], myManualSchema)

myDf.show()

+-----+----+-----+
| some| col|names|
+-----+----+-----+
|Hello|null|    1|
+-----+----+-----+



지금까지 DataFrame을 만드는 방법을 알아보았습니다. 
이제 가장 유용하게 사용하는 메서드를 알아보자.
* 컬럼이나 표현식을 사용하는 select 메서드
* 문자열 표현식을 사용하는 selectExpr 메서드
* 메서드로 사용할 수 없는 `org.apache.spark.sql.function` 패키지에 포함된 다양한 함수들
이 세가지 유형에 메서드로 DataFrame 을 다룰 때 필요한 대부분 트랜스포메이션 작업을 해결할 수 있다.

### select 와 selectExpr
select 와 selectExpr 메서드를 사용하면 데이터테이블에  SQL을 실행하는 것처럼 DataFrame에서도 SQL을 사용할 수 있다.
```
SELECT * FROM dataFrameTable
SELECT columnName FROM dataFrameTable
SELECT columnName * 10, otherColumn, someOtherCol as c FROM dataFrameTable
```


In [50]:
df.select("DEST_COUNTRY_NAME").show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [51]:
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows



In [60]:
from pyspark.sql.functions import expr, col,coalesce

df.select(
    expr("DEST_COUNTRY_NAME"),
    col("DEST_COUNTRY_NAME"),  
    coalesce("DEST_COUNTRY_NAME") ,
    "DEST_COUNTRY_NAME"
).show(3)

+-----------------+-----------------+---------------------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|coalesce(DEST_COUNTRY_NAME)|DEST_COUNTRY_NAME|
+-----------------+-----------------+---------------------------+-----------------+
|    United States|    United States|              United States|    United States|
|    United States|    United States|              United States|    United States|
|    United States|    United States|              United States|    United States|
+-----------------+-----------------+---------------------------+-----------------+
only showing top 3 rows



column 객체와 문자열을 함께 쓰는 실수를 많이 한다. 다음과 같은 예를 실행하면 컴파일러 오류가 발생한다. (안나는데)

In [56]:
df.select(col("DEST_COUNTRY_NAME"),  "DEST_COUNTRY_NAME").show(2)

+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+
|    United States|    United States|
|    United States|    United States|
+-----------------+-----------------+
only showing top 2 rows



`expr` 함수는 가장 유연한 참조 방식이다. `expr` 함수는 단순 컬럼 참조나 문자열을 이용해 컬럼 참조할 수 있습니다. 설명을 위해 AS 키워드로 컬럼명을 변경한 다음 alias 메서드로  변경해보겠습니다.

In [61]:
df.select(expr("DEST_COUNTRY_NAME AS destination")).show(2)

+-------------+
|  destination|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows



In [64]:
df.select(expr("DEST_COUNTRY_NAME AS destination").alias("temp")).show(2)

+-------------+
|         temp|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows



우리는 select 메서드에 expr 함수를 사용하는 패턴을 자주 사용합니다. 스파트는 이런 작업을 간단하고 효율적으로 활용할 수 있는 `selectExpr` 메서드를 제공합니다. selectExpr 메서드는 자주 사용하는 편리한 인터페이스중 하나이다.

In [65]:
df.selectExpr("DEST_COUNTRY_NAME as newColumnName", "DEST_COUNTRY_NAME").show(2)

+-------------+-----------------+
|newColumnName|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows



selectExpr 메서드는 새로운 DataFrame을 생성하는 복잡한 표현식을 간단하게 만드는 도구이다. 사실 모든 유효한  비집계형(non-aggregating) SQL 구문을 지정할 수 있다. 

In [67]:
# 다음 코드는 DataFrame에 출발지와 도착지가 같은지 나타내는 새로운 withinCountry 컬럼을 나타내는 예제이다.
df.selectExpr("*","(DEST_COUNTRY_NAME= ORIGIN_COUNTRY_NAME) as withinCountry" ).show(3)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
|    United States|            Ireland|  344|        false|
+-----------------+-------------------+-----+-------------+
only showing top 3 rows



select 표현식은 DataFrame의 컬럼에 대한 집계함수를 지정할 수 있습니다.

In [70]:
df.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show(22)

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



### 스파크 데이터 타입으로 변환하기
때로는 새로운 컬럼이 아닌 명시적인 값을 스파크에 전달해야합니다. 명시적인 값은 상수값일 수 있고, 추후에 비교에 사용할 무언가가 될 수 있습니다. 이 떄 **리터럴(literal)** 을 사용합니다. 리터럴은 프로그래밍 언어의 리터럴값을 스파크가 이해할 수 있는 값으로 변환합니다.

In [71]:
from pyspark.sql.functions import lit
df.select(expr("*"), lit(1).alias("One")).show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



어떤 상수나 프로그래밍으로 생성된 변숫값이 특정 컬럼값보다 큰지 확인할 때 리터럴을 사용합니다.

### 컬럼 추가하기 withColumn
DataFrame에 신규 컬럼을 추가하는 공식적인 방법은 DataFramedml withColumn 메서드를 사용하는 것이다. 숫자 1을 값으로  가지는 컬럼을 추가하는 예제는 다음과 같다.

In [72]:
df.withColumn("numberOne", lit(1)).show(2)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [73]:
df.withColumn("withinColumn", expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME")).show(1)

+-----------------+-------------------+-----+------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinColumn|
+-----------------+-------------------+-----+------------+
|    United States|            Romania|   15|       false|
+-----------------+-------------------+-----+------------+
only showing top 1 row



withColumn 메소드는 두 개의 인수를 사용합니다. 하나는 컬럼명이고, 다른 하나는 값을 생성할 표현식입니다. 한 가지 재밌는 것은 withColumn 메서드로  컬럼명을 변경할 수 있단 것입니다. 

In [74]:
df.withColumn("withinColumn",expr("ORIGIN_COUNTRY_NAME")).columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count', 'withinColumn']

### 컬럼명 변경하기
withColumn 메서드 대신 withColumnRenamed 메서드로 컬럼명을 변경할 수 있다. `withColumnRenamed`메서드는 첫 번째 인수로 전달된 컬럼명을 두 번쨰 인수의 문자열로 변경한다.

In [76]:
df.withColumnRenamed("DEST_COUNTRY_NAME", "dest").columns

['dest', 'ORIGIN_COUNTRY_NAME', 'count']

### 예약 문자와 키워드
공백이나 하이픈(-) 같은 예약 문자는 컬럼명에 사용할 수 없다.  예약 문자를 컬럼명에 사용하려면 백틱(\`) 문자를 사용해 이스케이프(escape)해야한다. withColumn 메서드를 사용해 예약 문자가 포함된 컬럼을 생성해보자. 다음은 이스케이핑 문자가 필요한 경우와 필요없는 경우 예제이다.

In [78]:
dfWithLongColName = df.withColumn(
"This Long Column Name", expr("ORIGIN_COUNTRY_NAME"))
# 위 예제에서는 withColumn 메서드의 첫 번째 인수로 새로운 컬럼명을 나타내는 문자열을 지정했기때문에 이스케이프 (escape) 문자가 필요없다. 

In [80]:
dfWithLongColName.selectExpr("`This Long Column Name`", "`This Long Column Name` as `newTemp small col`").show(2)

+---------------------+-----------------+
|This Long Column Name|newTemp small col|
+---------------------+-----------------+
|              Romania|          Romania|
|              Croatia|          Croatia|
+---------------------+-----------------+
only showing top 2 rows



표현식 대신 문자열을 사용해 명시적으로 컬럼을 참조하면 리터럴로 해석되기 때문에 예약 문자가 포함된 컬럼을 참조할 수 있다. 예약 문자나 키워드를 사용하는 표현식에만 이스케이프 처리가 필요하다. 다음 두 예제 모두 같은 DataFrame을 만든다.

In [82]:
dfWithLongColName.select(expr("`This Long Column Name`")).columns

['This Long Column Name']

### 대소문자 구분
기본적으로 스파크는 대소문자를 가리지 않는다. 다음과 같은 설정을 통해 스파크에서 대소문자를 구분하게 만들 수 있다.
```
set spark.sql.caseSensitive true


```

### 컬럼 제거하기
DataFrame 에서 컬럼을 제거하는 방법을 알아보자. select 메서드로 컬럼을 제거할 수 있지만, 컬럼을 제거하는 메서드인 drop을 이용할 수 있다. 

In [83]:
df.drop("ORIGIN_COUNTRY_NAME").columns


['DEST_COUNTRY_NAME', 'count']

다수의 컬럼명을 drop 메서드의 인수로 사용해 컬럼을 한꺼번에 제거할 수 있다.

In [84]:
df.drop("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").columns


['count']

### 컬럼의 데이터타입 변경하기
`cast` 메서드로 데이터타입을 변환할 수 있다. 

In [85]:
df.withColumn("count2", col("count").cast("string"))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint, count2: string]

####  필터링 : where , filter

In [86]:
df.filter(col("count")> 2).show(1)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
+-----------------+-------------------+-----+
only showing top 1 row



In [88]:
df.where("count> 2").show(1)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
+-----------------+-------------------+-----+
only showing top 1 row



같은 표현식에 여러 필터를 적용해야 할 때도 있습니다. 하지만 Spark 는 자동으로 필터의 순서와 상관없이 동시에 모든 필터링 작업을 수행하기 때문에 항상 유요한건 아닙니다. 그러므로 여러 개의 AND 필터를 지정하려면 차례대로 필터를 연결하고 판단은 Spark 에 맡겨야합니다.

In [89]:
df.where(col("count")<2).where(col("ORIGIN_COUNTRY_NAME")!="Croatia").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



### 고유한 로우 얻기
`distinct` 메서드는 중복되지 않은 로우를 가진 신규 DataFrame을 반환한다.

In [90]:
df.select("ORIGIN_COUNTRY_NAME","DEST_COUNTRY_NAME" ).distinct().count()

256

In [91]:
df.select("ORIGIN_COUNTRY_NAME").distinct().count()

125

### 무작위 샘플 만들기
* `sample` 메서드를 이용하면 된다.
* DataFrame에서 표본 데이터 추출 비율을 지정할 수 있고, 복원 추출이나 비복원 추출의 사용 여부를 지정할 수 있다.

In [94]:
seed = 5
withReplacement=False
fraction = 0.5
df.sample(withReplacement, fraction, seed).count()

138

### 임의 분할하기
임의 분할은 DataFrame을 임의 크기로 "분할"할 때 유용하게 사용됩니다. 이 기능은 머신러닝 알고리즘에 사용할 학습셋, 검증셋, 그리고 테스트 셋을 만들 때 주로 사용합니다. 다음 예제에서는 분할 가중치를 함수의 파라미터로 설정해 원본 DataFrame을 서로 다른 데이터를 가진 두 개의 DataFrame으로 나눕니다. 이 메서드는 임의성(randomized)을 가지도록 설계되어 시드값을 반드시 설정해야합니다. 총합이 1이 되도록 각 DataFrame 비율을 지정하지 않으면 예제와 같은 비율로 지정됩니다.

In [95]:
dataFrames = df.randomSplit([0.25, 0.75], seed)
dataFrames[0].count()> dataFrames[1].count()

False

### 로우 합치기와 추가하기
DataFrame에 레코드를 추가하려면 원본 DataFrame에 새로운 DataFrame을 통합해야한다. 통합은 두 개의 DataFrameㅡㅇ ㄹ단순히 결합하는 행위이다. 통합하려는 두 개의 DataFrame은 반드시 동일한 스키마와 컬럼 수를 가져야한다.
> union 메서드는 현재 스키마가 아닌 컬럼 위치를 기반으로 동작합니다.

In [97]:
from pyspark.sql import Row
schema = df.schema
newRows=[
    Row("New Country", "Other Country", 4),
    Row("New Country", "Other Country2", 1)
]

parallelizedRows = spark.sparkContext.parallelize(newRows)
newDF= spark.createDataFrame(parallelizedRows , schema)
df.union(newDF).where("count=1").where(col("ORIGIN_COUNTRY_NAME")!="United States").show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|    United States|          Gibraltar|    1|
|    United States|             Cyprus|    1|
|    United States|            Estonia|    1|
|    United States|          Lithuania|    1|
|    United States|           Bulgaria|    1|
|    United States|            Georgia|    1|
|    United States|            Bahrain|    1|
|    United States|   Papua New Guinea|    1|
|    United States|         Montenegro|    1|
|    United States|            Namibia|    1|
|      New Country|     Other Country2|    1|
+-----------------+-------------------+-----+



### 로우 정렬하기
sort 와 orderBy 메서드를 사용해 DataFrame의 최댓값 혹은 최솟값이 상단에  위치하도록 정렬할 수 있다. 두 메서드는 완벽히 같게 동작한다. 두 메서드 모두 컬럼 표현식과 문자열을 사용할 수 있고, 다수의 컬럼을 지정할 수 있다. 기본 동작은 오름차순이다.

In [99]:
df.sort("count").show(233)

+--------------------+--------------------+-----+
|   DEST_COUNTRY_NAME| ORIGIN_COUNTRY_NAME|count|
+--------------------+--------------------+-----+
|       United States|            Bulgaria|    1|
|       United States|             Georgia|    1|
|               Malta|       United States|    1|
|       United States|              Cyprus|    1|
|       United States|             Bahrain|    1|
|       United States|           Singapore|    1|
|       Cote d'Ivoire|       United States|    1|
|        Burkina Faso|       United States|    1|
|       United States|    Papua New Guinea|    1|
|              Kosovo|       United States|    1|
|       United States|           Gibraltar|    1|
|            Djibouti|       United States|    1|
|                Iraq|       United States|    1|
|           Indonesia|       United States|    1|
|       United States|             Estonia|    1|
|       New Caledonia|       United States|    1|
|       United States|          Montenegro|    1|


In [100]:
df.orderBy("count", "DEST_COUNTRY_NAME").show(2000)

+--------------------+--------------------+------+
|   DEST_COUNTRY_NAME| ORIGIN_COUNTRY_NAME| count|
+--------------------+--------------------+------+
|        Burkina Faso|       United States|     1|
|       Cote d'Ivoire|       United States|     1|
|              Cyprus|       United States|     1|
|            Djibouti|       United States|     1|
|           Indonesia|       United States|     1|
|                Iraq|       United States|     1|
|              Kosovo|       United States|     1|
|               Malta|       United States|     1|
|             Moldova|       United States|     1|
|       New Caledonia|       United States|     1|
|Saint Vincent and...|       United States|     1|
|            Suriname|       United States|     1|
|       United States|             Estonia|     1|
|       United States|              Cyprus|     1|
|       United States|           Lithuania|     1|
|       United States|            Bulgaria|     1|
|       United States|         

In [101]:
df.orderBy(col("count"), col("DEST_COUNTRY_NAME")).show(2000)

+--------------------+--------------------+------+
|   DEST_COUNTRY_NAME| ORIGIN_COUNTRY_NAME| count|
+--------------------+--------------------+------+
|        Burkina Faso|       United States|     1|
|       Cote d'Ivoire|       United States|     1|
|              Cyprus|       United States|     1|
|            Djibouti|       United States|     1|
|           Indonesia|       United States|     1|
|                Iraq|       United States|     1|
|              Kosovo|       United States|     1|
|               Malta|       United States|     1|
|             Moldova|       United States|     1|
|       New Caledonia|       United States|     1|
|Saint Vincent and...|       United States|     1|
|            Suriname|       United States|     1|
|       United States|             Estonia|     1|
|       United States|              Cyprus|     1|
|       United States|           Lithuania|     1|
|       United States|            Bulgaria|     1|
|       United States|         

정렬 기준을 명확히 지정하려면 asc나 desc 함수를 사용한다.

In [102]:
from pyspark.sql.functions import desc, asc
df.orderBy(expr("count dessc")).show(2)
df.orderBy(col("count").desc(), col("DEST_COUNTRY_NAME").asc()).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Moldova|      United States|    1|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+------------------+-------------------+------+
| DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+------------------+-------------------+------+
|     United States|      United States|370002|
|     United States|             Canada|  8483|
|            Canada|      United States|  8399|
|     United States|             Mexico|  7187|
|            Mexico|      United States|  7140|
|    United Kingdom|      United States|  2025|
|     United States|     United Kingdom|  1970|
|             Japan|      United States|  1548|
|     United States|              Japan|  1496|
|           Germany|      United States|  1468|
|     United States| Dominican Republic|  1420|
|Dominican Republic|      U

asc_nulls_first, desc_nulls_first, asc_nulls_last, desc_nulls_last 메서드를 사용해 DataFrame에서 null 값이 표시되는 기준을 정할 수 있다.

트랜스포메이션을 처리하기 전에 성능을 최적화하기 위해 파티션별 정렬을 수행하기도 한다. 파티션별 정렬은 sortWithinPartitions 메소드로 할 수 이다.


In [104]:
spark.read.format("json").load("./data/flight-data/json/*-summary.json").sortWithinPartitions("count")

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

### 로우 수 제한하기
`limit` 메서드를 이용해 추출 로우수를 제한

In [107]:
df.orderBy(expr("count desc")).limit(5).show(10,False)

+--------------------------------+-------------------+-----+
|DEST_COUNTRY_NAME               |ORIGIN_COUNTRY_NAME|count|
+--------------------------------+-------------------+-----+
|Malta                           |United States      |1    |
|Saint Vincent and the Grenadines|United States      |1    |
|United States                   |Croatia            |1    |
|United States                   |Gibraltar          |1    |
|United States                   |Singapore          |1    |
+--------------------------------+-------------------+-----+



### repartition과 coalesce
또다른 최적화 기법은 자주 필터링하는 컬럼을 기준으로 데이터를 분할하는 것이다. 이를 통해 파티셔닝 스키마와 파티션 수를 포함해 클러스터 전반의 물리적 데이터 구성을 제어할 수 있다.

`repartition` 메서드를 호출하면 무조건 전체 데이터를 셔플한다. 
향후에 사용할 파티션 수가 현재 파티션 수보다 많거나 컬럼을 기준으로 파티션을 만드는 경우에만 사용한다.

In [109]:
df.rdd.getNumPartitions()


1

In [110]:
df.repartition(5)

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

특정 컬럼을 기준으로 자주 필터링하면 자주 필터링되는 컬럼을 기준으로 파티션을 재분배하는 것이 좋다.

In [111]:
df.repartition(col("DEST_COUNTRY_NAME"))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

선택적으로 파티션 수를 지정할 수 있다.

In [112]:
df.repartition(5, col("DEST_COUNTRY_NAME"))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

`coalesce` 메서드는 전체 데이터를 셔플하지 않고 파티션을 병합하려는 경우에 사용한다. 다음은 목적지를 기준으로 셔플을 수행해 5개의 파티션으로 나누고, 전체 데이터를 셔플없이 병합하는 예제이다.

In [113]:

df.repartition(5, col("DEST_COUNTRY_NAME")).coalesce(2)

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

### 드라이버로 로우 데이터 수집하기
스파크는 드라이버에서 클러스터 상태 정보를 유지한다. 로컬 환경에서 데이터를 다루려면 드라이버로 데이털 수집해야한다.

* collect 메서드는 전체 DataFrame의 모든 데이터를 수집하고
* take 메서드는 상위 N 개의 로우를 반환한다.
* show 메서드는 여러 로우를 보기 좋게 출력한다.

In [114]:
collectDF = df.limit(10)
collectDF.take(5) # take 는 정수형 값을 인수로 사용한다.

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)]

In [115]:
collectDF.show() # 정돈된 형태로 출력한다.

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+



In [116]:
collectDF.show(3,False)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
+-----------------+-------------------+-----+
only showing top 3 rows



In [117]:
collectDF.collect()

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

전체 데이터셋에 대한 반복 처리를 위해 드라이버로 로우를 모으는 또다른 방법 `toLocalIterator` 메서드는 이터레이터로 모든 파티션의 데이터를 드라이버에 전달한다. toLLocalIterator 메서드를 사용해 데이터셋의 파티션을 차례로 반복 처리를 할 수 있다.

In [118]:
collectDF.toLocalIterator()

<generator object _local_iterator_from_socket.<locals>.PyLocalIterable.__iter__ at 0x110c6fc80>

> 드라이버로 모든 데이터 컬렉션을 수집하는 작업은 매우 큰 비용 ( CPU, 메모리, 네트워크 등 ) 이 발생한다. 대규모 데이터셋에 `collect` 명령을 수행하면 드라이버가 비정상적으로 종료될 수 있다.`toLocalIterator` 메서드도 이다.`toLocalIterator`메서드를 사용할 때, 매우 큰 파티션이 있다면 드라이버와 애플리케이션이 비정상적으로 종료될 수 있다. 또한 연산을 병렬로 수행하지 않고 차례로 처리해 매우 큰 처리 비용이 발생한다.